In [ ]:
from google.colab import drive
#drive.flush_and_unmount()
drive.mount('/content/drive')
#drive.mount('/content/drive', force_remount=True)

In [11]:
!pip install transformers
!pip install sentencepiece
!pip install tensorflow==2.7.0
!pip install stanza
!pip install transformers
!pip install tensorflow-addons
!pip install nltk
!pip install textacy

In [1]:
import tensorflow as tf
print(tf.__version__)

2.7.0


In [8]:
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
from sklearn.model_selection import train_test_split
from textacy.datasets.supreme_court import SupremeCourt
import numpy as np
import re
import unicodedata
import nltk
#from transformers import pipeline
from nltk.corpus import stopwords
from tensorflow import keras
from tensorflow.keras.layers import Dense,Dropout, Input, BatchNormalization
from tqdm import tqdm
import pickle
from sklearn.metrics import confusion_matrix,f1_score,classification_report
import matplotlib.pyplot as plt
import itertools
from sklearn.utils import shuffle
from tensorflow.keras import regularizers
#from transformers import *
from transformers import BertTokenizer, TFBertModel, BertConfig,TFDistilBertModel,DistilBertTokenizer,DistilBertConfig
import pandas as pd
from transformers import AutoTokenizer, TFAutoModel
import numpy as np
import gc
import math
import json
import stanza
from tensorflow.keras import *
import tensorflow as tf
from tensorflow.keras import *
import tensorflow.keras.backend as K
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report
from transformers import TFRobertaModel,RobertaTokenizer,LongformerTokenizer,TFLongformerModel
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.initializers import RandomUniform

from numpy.random import seed
import random as python_random
import os
import sys
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

np.random.seed(1)
python_random.seed(1)
tf.random.set_seed(1)

In [ ]:
!cp "/content/drive/My Drive/labels_sc.txt" "./labels_sc.txt"
!cp "/content/drive/My Drive/labels_sc_279.txt" "./labels_sc_279.txt"

In [9]:
# Create a MirroredStrategy.
strategy = tf.distribute.MirroredStrategy()
print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')
Number of devices: 2


In [10]:

def create_model():
    inps = Input(shape = (max_len,), dtype='int64')
    masks= Input(shape = (max_len,), dtype='int64')
    dbert_layer = dbert_model(inps, attention_mask=masks)[0][:,0,:]
    dense_0 = Dense(512,activation='relu',kernel_regularizer=regularizers.l2(0.01))(dbert_layer)
    dropout_0= Dropout(0.5)(dense_0)
    pred = Dense(279, activation='softmax',kernel_regularizer=regularizers.l2(0.01))(dropout_0)
    model = tf.keras.Model(inputs=[inps,masks], outputs=pred)
    print(model.summary())
    return model   

In [5]:
with strategy.scope():
    total_accuracy=0
    total_weighted_f1=0
    total_micro_f1=0
    total_weighted_precision=0
    total_micro_precision=0
    total_weighted_recall=0
    total_micro_recall=0
    sc = SupremeCourt()
    print(sc.info)
    sc.download()

    texts = []  # list of text samples
    labels_index = {}  # dictionary mapping label name to numeric id
    labels = []  # list of label ids

    issue_codes = list(sc.issue_area_codes.keys()) # 15 labels
    print(issue_codes)
    issue_codes.sort()
    issue_codes = [str(ic) for ic in issue_codes]

    labels_index = dict(zip(issue_codes, np.arange(len(issue_codes))))
    print(labels_index)
    count=0

    for record in sc.records():

            count=count+1
            if record[1]['issue'] == None: # some cases have None as an issue
                labels.append(labels_index['-1'])
            else:
                labels.append(labels_index[record[1]['issue'][:-4]])

            new_sen=record[0].split("Footnotes")[0]
            new_new_sen=new_sen.split()
            if len(new_new_sen) >= 4096:
              new_new_sen=new_new_sen[0:4096]

            elif len(new_new_sen) < 4096:
              new_new_sen=new_new_sen[0:len(new_new_sen)]

            new_new_sen=' '.join(new_new_sen)

            texts.append(new_new_sen)

    len_list = [len(ele.split()) for ele in texts]

    #print(labels)
    print(len(labels))

    res = 0 if len(len_list) == 0 else (float(sum(len_list)) / len(len_list))
    print("Average Length %s" % res) 
    print('Found %s texts.' % len(texts))
    print('Found %s labels.' % len(labels_index))
    temp_file = open("labels_sc_279.txt", "r")

    data = temp_file.read()
    label_list = data.split("\n")
    #print(label_list)
    label_list = label_list[0:-1]
    #print(label_list)
    label_list = [int(i) for i in label_list]
    summarized_data = pd.DataFrame(texts,
                   columns =['text'])
    summarized_data['label'] = label_list
    print(summarized_data)
    #gc.collect()
    #tf.keras.backend.clear_session()
    dbert_tokenizer = LongformerTokenizer.from_pretrained('saibo/legal-longformer-base-4096')
    dbert_model = TFLongformerModel.from_pretrained('saibo/legal-longformer-base-4096')
    max_len=4096
    sentences=summarized_data['text']
    labels=summarized_data['label']
    len(sentences),len(labels)
    model_0=create_model()
    input_ids=[]
    attention_masks=[]

    for sent in sentences:
      dbert_inps=dbert_tokenizer.encode_plus(sent,add_special_tokens = True,max_length =max_len,pad_to_max_length = True,return_attention_mask = True,truncation=True)
      input_ids.append(dbert_inps['input_ids'])
      attention_masks.append(dbert_inps['attention_mask'])
    input_ids=np.asarray(input_ids)

    attention_masks=np.array(attention_masks)
    labels=np.array(labels)
    train_inp,val_inp,train_label,val_label,train_mask,val_mask=train_test_split(input_ids,labels,attention_masks,test_size=0.1,random_state=42)
    print(train_inp.shape)
    print(train_mask.shape)
    print(train_label.shape)
    print(val_inp.shape)
    print(val_mask.shape)
    print(val_label.shape)
    train_inp=train_inp[0:7576]
    train_mask=train_mask[0:7576]
    train_label=train_label[0:7576]

    val_inp=val_inp[0:840]
    val_label=val_label[0:840]
    val_mask=val_mask[0:840]
    print(train_inp.shape)
    print(train_mask.shape)
    print(train_label.shape)
    print(val_inp.shape)
    print(val_mask.shape)
    print(val_label.shape)
    
    log_dir='dbert_model'

    model_save_path='LSM/legallongformer-0-279labels.h5'


    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    accuracy = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')

    optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
    callbacks= [tf.keras.callbacks.ModelCheckpoint(filepath=model_save_path,monitor='val_accuracy',mode='max',save_best_only=True,save_weights_only=True),keras.callbacks.TensorBoard(log_dir=log_dir)]
    model_0.compile(loss=loss,optimizer=optimizer, metrics=[accuracy])
          

     
    

{'name': 'supreme_court', 'site_url': 'http://caselaw.findlaw.com/court/us-supreme-court', 'description': 'Collection of ~8.4k decisions issued by the U.S. Supreme Court between November 1946 and June 2016.'}
[-1, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
{'-1': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9, '10': 10, '11': 11, '12': 12, '13': 13, '14': 14}
8419
Average Length 2920.3854377004395
Found 8419 texts.
Found 15 labels.
                                                   text  label
0     [ Halliburton Oil Well Cementing Co. v. Walker...    209
1     Rehearing Denied Dec. 16, 1946. See . Mr.Claud...     63
2     Rehearing Denied Dec. 16, 1946 See . Appeal fr...    216
3     Mr. Walter J. Cummings, Jr., of Washington, D....    108
4     Mr.A. Devitt Vaneck, of Washington, D.C., for ...    196
...                                                 ...    ...
8414  Opinion reported: Ante, p. 88. DECREE 1. It is...     74
8415  In this dispute between

2022-09-16 00:48:41.075072: I tensorflow/stream_executor/cuda/cuda_blas.cc:1774] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
Some layers from the model checkpoint at saibo/legal-longformer-base-4096 were not used when initializing TFLongformerModel: ['lm_head']
- This IS expected if you are initializing TFLongformerModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFLongformerModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFLongformerModel were initialized from the model checkpoint at saibo/legal-longformer-base-4096.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFLongf

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 4096)]       0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 4096)]       0           []                               
                                                                                                  
 tf_longformer_model (TFLongfor  TFLongformerBaseMod  148659456  ['input_1[0][0]',                
 merModel)                      elOutputWithPooling               'input_2[0][0]']                
                                (last_hidden_state=                                               
                                (None, 4096, 768),                                            

/ext3/miniconda3/envs/wol_env/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:2291: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


(7577, 4096)
(7577, 4096)
(7577,)
(842, 4096)
(842, 4096)
(842,)
(7576, 4096)
(7576, 4096)
(7576,)
(840, 4096)
(840, 4096)
(840,)
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job

In [11]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
    print('Not connected to a GPU')
else:
    print(gpu_info)
  

Fri Sep 16 00:55:01 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  On   | 00000000:31:00.0 Off |                    0 |
| N/A   27C    P0    71W / 500W |  79810MiB / 81920MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA A100-SXM...  On   | 00000000:CA:00.0 Off |                    0 |
| N/A   

In [19]:
gc.collect()
tf.keras.backend.clear_session()
history=model_0.fit([train_inp,train_mask],train_label,batch_size=8,epochs=5,validation_data=([val_inp,val_mask],val_label),callbacks=callbacks)

2022-09-15 02:03:34.542831: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:766] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Did not find a shardable source, walked to a node which is not a dataset: name: "FlatMapDataset/_9"
op: "FlatMapDataset"
input: "PrefetchDataset/_8"
attr {
  key: "Targuments"
  value {
    list {
    }
  }
}
attr {
  key: "_cardinality"
  value {
    i: -2
  }
}
attr {
  key: "f"
  value {
    func {
      name: "__inference_Dataset_flat_map_slice_batch_indices_100388"
    }
  }
}
attr {
  key: "metadata"
  value {
    s: "\n\021FlatMapDataset:36"
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: 8
        }
      }
    }
  }
}
attr {
  key: "output_types"
  value {
    list {
      type: DT_INT64
    }
  }
}
. Consider either turning off auto-sharding or switching the auto_shard_policy to DATA to shard this dataset. Yo

Epoch 1/5


/ext3/miniconda3/envs/wol_env/lib/python3.7/site-packages/tensorflow/python/util/dispatch.py:1096: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


INFO:tensorflow:batch_all_reduce: 270 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:GPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1').
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:GPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1').
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:GPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1').
INFO:tensorflow:batch_all_reduce: 270 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:GPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1').
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:GPU:0 then br

2022-09-15 02:19:59.806305: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:766] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Did not find a shardable source, walked to a node which is not a dataset: name: "FlatMapDataset/_9"
op: "FlatMapDataset"
input: "PrefetchDataset/_8"
attr {
  key: "Targuments"
  value {
    list {
    }
  }
}
attr {
  key: "_cardinality"
  value {
    i: -2
  }
}
attr {
  key: "f"
  value {
    func {
      name: "__inference_Dataset_flat_map_slice_batch_indices_239532"
    }
  }
}
attr {
  key: "metadata"
  value {
    s: "\n\021FlatMapDataset:61"
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: 8
        }
      }
    }
  }
}
attr {
  key: "output_types"
  value {
    list {
      type: DT_INT64
    }
  }
}
. Consider either turning off auto-sharding or switching the auto_shard_policy to DATA to shard this dataset. Yo

947/947 [==============================] - 1049s 953ms/step - loss: 12.4965 - accuracy: 0.2051 - val_loss: 10.3824 - val_accuracy: 0.3774
Epoch 2/5
947/947 [==============================] - 873s 922ms/step - loss: 9.6101 - accuracy: 0.4219 - val_loss: 8.7146 - val_accuracy: 0.4857
Epoch 3/5
947/947 [==============================] - 873s 922ms/step - loss: 8.1805 - accuracy: 0.5149 - val_loss: 7.7446 - val_accuracy: 0.5238
Epoch 4/5
947/947 [==============================] - 874s 923ms/step - loss: 7.0716 - accuracy: 0.5948 - val_loss: 7.0143 - val_accuracy: 0.5464
Epoch 5/5
947/947 [==============================] - 873s 922ms/step - loss: 6.1665 - accuracy: 0.6639 - val_loss: 6.5217 - val_accuracy: 0.5524


In [12]:
pred_labels=[]

loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
accuracy = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)

model_saved= create_model()
model_saved.compile(loss=loss,optimizer=optimizer, metrics=[accuracy])
model_saved.load_weights('LSM/legallongformer-0-279labels.h5')


print(val_inp.shape)
print(val_mask.shape)
print(val_label.shape)
eval_loss, eval_acc=model_saved.evaluate([val_inp,val_mask],val_label,batch_size=8)
print(eval_acc)


for i in range(0,len(val_inp)):
    pred=model_saved.predict([val_inp[i].reshape(1,4096),val_mask[i].reshape(1,4096)],batch_size=1)
    pred_label = pred.argmax(axis=1)
    pred_labels.append(pred_label)
accuracy=accuracy_score(val_label, pred_labels)
print("Accuracy: "+str(accuracy))
total_accuracy=total_accuracy+accuracy

weighted_f1=f1_score(val_label,pred_labels, average='weighted')
print("Weighted F1: "+ str(weighted_f1))
total_weighted_f1=total_weighted_f1+weighted_f1
micro_f1=f1_score(val_label,pred_labels, average='micro')
print("Micro F1: "+ str(micro_f1))
total_micro_f1=total_micro_f1+micro_f1

weighted_precision=precision_score(val_label, pred_labels, average='weighted')
print("Weighted Precision: " + str(weighted_precision))
total_weighted_precision=total_weighted_precision+weighted_precision
micro_precision=precision_score(val_label, pred_labels, average='micro')
print("Micro Precision: " + str(micro_precision))
total_micro_precision=total_micro_precision+micro_precision

weighted_recall=recall_score(val_label, pred_labels, average='weighted')
print("Weighted Recall: " + str(weighted_recall))
total_weighted_recall=total_weighted_recall+weighted_recall
micro_recall=recall_score(val_label, pred_labels, average='micro')
print("Micro Recall: " + str(micro_recall))
total_micro_recall=total_micro_recall+micro_recall

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 4096)]       0           []                               
                                                                                                  
 input_6 (InputLayer)           [(None, 4096)]       0           []                               
                                                                                                  
 tf_longformer_model (TFLongfor  TFLongformerBaseMod  148659456  ['input_5[0][0]',                
 merModel)                      elOutputWithPooling               'input_6[0][0]']                
                                (last_hidden_state=                                               
                                (None, 4096, 768),                                          

/ext3/miniconda3/envs/wol_env/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/ext3/miniconda3/envs/wol_env/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
with strategy.scope():
    sc = SupremeCourt()
    print(sc.info)
    sc.download()

    texts = []  # list of text samples
    labels_index = {}  # dictionary mapping label name to numeric id
    labels = []  # list of label ids

    issue_codes = list(sc.issue_area_codes.keys()) # 15 labels
    print(issue_codes)
    issue_codes.sort()
    issue_codes = [str(ic) for ic in issue_codes]

    labels_index = dict(zip(issue_codes, np.arange(len(issue_codes))))
    print(labels_index)
    count=0

    for record in sc.records():

            count=count+1
            if record[1]['issue'] == None: # some cases have None as an issue
                labels.append(labels_index['-1'])
            else:
                labels.append(labels_index[record[1]['issue'][:-4]])

            new_sen=record[0].split("Footnotes")[0]
            new_new_sen=new_sen.split()
            if len(new_new_sen) >= 4096:
              new_new_sen=new_new_sen[0:4096]

            elif len(new_new_sen) < 4096:
              new_new_sen=new_new_sen[0:len(new_new_sen)]

            new_new_sen=' '.join(new_new_sen)

            texts.append(new_new_sen)

    len_list = [len(ele.split()) for ele in texts]

    #print(labels)
    print(len(labels))

    res = 0 if len(len_list) == 0 else (float(sum(len_list)) / len(len_list))
    print("Average Length %s" % res) 
    print('Found %s texts.' % len(texts))
    print('Found %s labels.' % len(labels_index))
    temp_file = open("labels_sc_279.txt", "r")

    data = temp_file.read()
    label_list = data.split("\n")
    #print(label_list)
    label_list = label_list[0:-1]
    #print(label_list)
    label_list = [int(i) for i in label_list]
    summarized_data = pd.DataFrame(texts,
                   columns =['text'])
    summarized_data['label'] = label_list
    print(summarized_data)
    #gc.collect()
    #tf.keras.backend.clear_session()
    dbert_tokenizer = LongformerTokenizer.from_pretrained('saibo/legal-longformer-base-4096')
    dbert_model = TFLongformerModel.from_pretrained('saibo/legal-longformer-base-4096')
    max_len=4096
    sentences=summarized_data['text']
    labels=summarized_data['label']
    len(sentences),len(labels)
    model_0=create_model()
    input_ids=[]
    attention_masks=[]

    for sent in sentences:
      dbert_inps=dbert_tokenizer.encode_plus(sent,add_special_tokens = True,max_length =max_len,pad_to_max_length = True,return_attention_mask = True,truncation=True)
      input_ids.append(dbert_inps['input_ids'])
      attention_masks.append(dbert_inps['attention_mask'])
    input_ids=np.asarray(input_ids)

    attention_masks=np.array(attention_masks)
    labels=np.array(labels)
    train_inp,val_inp,train_label,val_label,train_mask,val_mask=train_test_split(input_ids,labels,attention_masks,test_size=0.1,random_state=42)
    print(train_inp.shape)
    print(train_mask.shape)
    print(train_label.shape)
    print(val_inp.shape)
    print(val_mask.shape)
    print(val_label.shape)
    train_inp=train_inp[0:7576]
    train_mask=train_mask[0:7576]
    train_label=train_label[0:7576]

    val_inp=val_inp[0:840]
    val_label=val_label[0:840]
    val_mask=val_mask[0:840]
    print(train_inp.shape)
    print(train_mask.shape)
    print(train_label.shape)
    print(val_inp.shape)
    print(val_mask.shape)
    print(val_label.shape)
    log_dir='dbert_model'

    model_save_path='LSM/legallongformer-1-279labels.h5'

    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    accuracy = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')

    optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
    callbacks= [tf.keras.callbacks.ModelCheckpoint(filepath=model_save_path,monitor='val_accuracy',mode='max',save_best_only=True,save_weights_only=True),keras.callbacks.TensorBoard(log_dir=log_dir)]
    model_0.compile(loss=loss,optimizer=optimizer, metrics=[accuracy])

In [ ]:
gc.collect()
tf.keras.backend.clear_session()
history=model_0.fit([train_inp,train_mask],train_label,batch_size=8,epochs=5,validation_data=([val_inp,val_mask],val_label),callbacks=callbacks)

2022-09-15 03:25:23.968603: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:766] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Did not find a shardable source, walked to a node which is not a dataset: name: "FlatMapDataset/_9"
op: "FlatMapDataset"
input: "PrefetchDataset/_8"
attr {
  key: "Targuments"
  value {
    list {
    }
  }
}
attr {
  key: "_cardinality"
  value {
    i: -2
  }
}
attr {
  key: "f"
  value {
    func {
      name: "__inference_Dataset_flat_map_slice_batch_indices_449347"
    }
  }
}
attr {
  key: "metadata"
  value {
    s: "\n\023FlatMapDataset:9371"
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: 8
        }
      }
    }
  }
}
attr {
  key: "output_types"
  value {
    list {
      type: DT_INT64
    }
  }
}
. Consider either turning off auto-sharding or switching the auto_shard_policy to DATA to shard this dataset. 

Epoch 1/5


/ext3/miniconda3/envs/wol_env/lib/python3.7/site-packages/tensorflow/python/util/dispatch.py:1096: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


691/947 [====================>.........] - ETA: 3:45 - loss: 13.0110 - accuracy: 0.1702

In [13]:
pred_labels=[]

loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
accuracy = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)

model_saved= create_model()
model_saved.compile(loss=loss,optimizer=optimizer, metrics=[accuracy])
model_saved.load_weights('LSM/legallongformer-1-279labels.h5')

for i in range(0,len(val_inp)):
    pred=model_saved.predict([val_inp[i].reshape(1,4096),val_mask[i].reshape(1,4096)],batch_size=8)
    pred_label = pred.argmax(axis=1)
    pred_labels.append(pred_label)
accuracy=accuracy_score(val_label, pred_labels)
print("Accuracy: "+str(accuracy))
total_accuracy=total_accuracy+accuracy

weighted_f1=f1_score(val_label,pred_labels, average='weighted')
print("Weighted F1: "+ str(weighted_f1))
total_weighted_f1=total_weighted_f1+weighted_f1
micro_f1=f1_score(val_label,pred_labels, average='micro')
print("Micro F1: "+ str(micro_f1))
total_micro_f1=total_micro_f1+micro_f1

weighted_precision=precision_score(val_label, pred_labels, average='weighted')
print("Weighted Precision: " + str(weighted_precision))
total_weighted_precision=total_weighted_precision+weighted_precision
micro_precision=precision_score(val_label, pred_labels, average='micro')
print("Micro Precision: " + str(micro_precision))
total_micro_precision=total_micro_precision+micro_precision

weighted_recall=recall_score(val_label, pred_labels, average='weighted')
print("Weighted Recall: " + str(weighted_recall))
total_weighted_recall=total_weighted_recall+weighted_recall
micro_recall=recall_score(val_label, pred_labels, average='micro')
print("Micro Recall: " + str(micro_recall))
total_micro_recall=total_micro_recall+micro_recall

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_7 (InputLayer)           [(None, 4096)]       0           []                               
                                                                                                  
 input_8 (InputLayer)           [(None, 4096)]       0           []                               
                                                                                                  
 tf_longformer_model (TFLongfor  TFLongformerBaseMod  148659456  ['input_7[0][0]',                
 merModel)                      elOutputWithPooling               'input_8[0][0]']                
                                (last_hidden_state=                                               
                                (None, 4096, 768),                                          

/ext3/miniconda3/envs/wol_env/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/ext3/miniconda3/envs/wol_env/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
with strategy.scope():
    sc = SupremeCourt()
    print(sc.info)
    sc.download()

    texts = []  # list of text samples
    labels_index = {}  # dictionary mapping label name to numeric id
    labels = []  # list of label ids

    issue_codes = list(sc.issue_area_codes.keys()) # 15 labels
    print(issue_codes)
    issue_codes.sort()
    issue_codes = [str(ic) for ic in issue_codes]

    labels_index = dict(zip(issue_codes, np.arange(len(issue_codes))))
    print(labels_index)
    count=0

    for record in sc.records():

            count=count+1
            if record[1]['issue'] == None: # some cases have None as an issue
                labels.append(labels_index['-1'])
            else:
                labels.append(labels_index[record[1]['issue'][:-4]])

            new_sen=record[0].split("Footnotes")[0]
            new_new_sen=new_sen.split()
            if len(new_new_sen) >= 4096:
              new_new_sen=new_new_sen[0:4096]

            elif len(new_new_sen) < 4096:
              new_new_sen=new_new_sen[0:len(new_new_sen)]

            new_new_sen=' '.join(new_new_sen)

            texts.append(new_new_sen)

    len_list = [len(ele.split()) for ele in texts]

    #print(labels)
    print(len(labels))

    res = 0 if len(len_list) == 0 else (float(sum(len_list)) / len(len_list))
    print("Average Length %s" % res) 
    print('Found %s texts.' % len(texts))
    print('Found %s labels.' % len(labels_index))
    temp_file = open("labels_sc_279.txt", "r")

    data = temp_file.read()
    label_list = data.split("\n")
    #print(label_list)
    label_list = label_list[0:-1]
    #print(label_list)
    label_list = [int(i) for i in label_list]
    summarized_data = pd.DataFrame(texts,
                   columns =['text'])
    summarized_data['label'] = label_list
    print(summarized_data)
    #gc.collect()
    #tf.keras.backend.clear_session()
    dbert_tokenizer = LongformerTokenizer.from_pretrained('saibo/legal-longformer-base-4096')
    dbert_model = TFLongformerModel.from_pretrained('saibo/legal-longformer-base-4096')
    max_len=4096
    sentences=summarized_data['text']
    labels=summarized_data['label']
    len(sentences),len(labels)
    model_0=create_model()
    input_ids=[]
    attention_masks=[]

    for sent in sentences:
      dbert_inps=dbert_tokenizer.encode_plus(sent,add_special_tokens = True,max_length =max_len,pad_to_max_length = True,return_attention_mask = True,truncation=True)
      input_ids.append(dbert_inps['input_ids'])
      attention_masks.append(dbert_inps['attention_mask'])
    input_ids=np.asarray(input_ids)

    attention_masks=np.array(attention_masks)
    labels=np.array(labels)
    train_inp,val_inp,train_label,val_label,train_mask,val_mask=train_test_split(input_ids,labels,attention_masks,test_size=0.1,random_state=42)
    print(train_inp.shape)
    print(train_mask.shape)
    print(train_label.shape)
    print(val_inp.shape)
    print(val_mask.shape)
    print(val_label.shape)
    train_inp=train_inp[0:7576]
    train_mask=train_mask[0:7576]
    train_label=train_label[0:7576]

    val_inp=val_inp[0:840]
    val_label=val_label[0:840]
    val_mask=val_mask[0:840]
    print(train_inp.shape)
    print(train_mask.shape)
    print(train_label.shape)
    print(val_inp.shape)
    print(val_mask.shape)
    print(val_label.shape)
    log_dir='dbert_model'

    model_save_path='LSM/legallongformer-2-279labels.h5'

    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    accuracy = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')

    optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
    callbacks= [tf.keras.callbacks.ModelCheckpoint(filepath=model_save_path,monitor='val_accuracy',mode='max',save_best_only=True,save_weights_only=True),keras.callbacks.TensorBoard(log_dir=log_dir)]
    model_0.compile(loss=loss,optimizer=optimizer, metrics=[accuracy])

In [ ]:
gc.collect()
tf.keras.backend.clear_session()
history=model_0.fit([train_inp,train_mask],train_label,batch_size=8,epochs=5,validation_data=([val_inp,val_mask],val_label),callbacks=callbacks)

In [14]:
pred_labels=[]

loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
accuracy = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)

model_saved= create_model()
model_saved.compile(loss=loss,optimizer=optimizer, metrics=[accuracy])
model_saved.load_weights('LSM/legallongformer-2-279labels.h5')

for i in range(0,len(val_inp)):
    pred=model_saved.predict([val_inp[i].reshape(1,4096),val_mask[i].reshape(1,4096)],batch_size=8)
    pred_label = pred.argmax(axis=1)
    pred_labels.append(pred_label)
accuracy=accuracy_score(val_label, pred_labels)
print("Accuracy: "+str(accuracy))
total_accuracy=total_accuracy+accuracy

weighted_f1=f1_score(val_label,pred_labels, average='weighted')
print("Weighted F1: "+ str(weighted_f1))
total_weighted_f1=total_weighted_f1+weighted_f1
micro_f1=f1_score(val_label,pred_labels, average='micro')
print("Micro F1: "+ str(micro_f1))
total_micro_f1=total_micro_f1+micro_f1

weighted_precision=precision_score(val_label, pred_labels, average='weighted')
print("Weighted Precision: " + str(weighted_precision))
total_weighted_precision=total_weighted_precision+weighted_precision
micro_precision=precision_score(val_label, pred_labels, average='micro')
print("Micro Precision: " + str(micro_precision))
total_micro_precision=total_micro_precision+micro_precision

weighted_recall=recall_score(val_label, pred_labels, average='weighted')
print("Weighted Recall: " + str(weighted_recall))
total_weighted_recall=total_weighted_recall+weighted_recall
micro_recall=recall_score(val_label, pred_labels, average='micro')
print("Micro Recall: " + str(micro_recall))
total_micro_recall=total_micro_recall+micro_recall

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_9 (InputLayer)           [(None, 4096)]       0           []                               
                                                                                                  
 input_10 (InputLayer)          [(None, 4096)]       0           []                               
                                                                                                  
 tf_longformer_model (TFLongfor  TFLongformerBaseMod  148659456  ['input_9[0][0]',                
 merModel)                      elOutputWithPooling               'input_10[0][0]']               
                                (last_hidden_state=                                               
                                (None, 4096, 768),                                          

/ext3/miniconda3/envs/wol_env/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/ext3/miniconda3/envs/wol_env/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
with strategy.scope():
    sc = SupremeCourt()
    print(sc.info)
    sc.download()

    texts = []  # list of text samples
    labels_index = {}  # dictionary mapping label name to numeric id
    labels = []  # list of label ids

    issue_codes = list(sc.issue_area_codes.keys()) # 15 labels
    print(issue_codes)
    issue_codes.sort()
    issue_codes = [str(ic) for ic in issue_codes]

    labels_index = dict(zip(issue_codes, np.arange(len(issue_codes))))
    print(labels_index)
    count=0

    for record in sc.records():

            count=count+1
            if record[1]['issue'] == None: # some cases have None as an issue
                labels.append(labels_index['-1'])
            else:
                labels.append(labels_index[record[1]['issue'][:-4]])

            new_sen=record[0].split("Footnotes")[0]
            new_new_sen=new_sen.split()
            if len(new_new_sen) >= 4096:
              new_new_sen=new_new_sen[0:4096]

            elif len(new_new_sen) < 4096:
              new_new_sen=new_new_sen[0:len(new_new_sen)]

            new_new_sen=' '.join(new_new_sen)

            texts.append(new_new_sen)

    len_list = [len(ele.split()) for ele in texts]

    #print(labels)
    print(len(labels))

    res = 0 if len(len_list) == 0 else (float(sum(len_list)) / len(len_list))
    print("Average Length %s" % res) 
    print('Found %s texts.' % len(texts))
    print('Found %s labels.' % len(labels_index))
    temp_file = open("labels_sc_279.txt", "r")

    data = temp_file.read()
    label_list = data.split("\n")
    #print(label_list)
    label_list = label_list[0:-1]
    #print(label_list)
    label_list = [int(i) for i in label_list]
    summarized_data = pd.DataFrame(texts,
                   columns =['text'])
    summarized_data['label'] = label_list
    print(summarized_data)
    #gc.collect()
    #tf.keras.backend.clear_session()
    dbert_tokenizer = LongformerTokenizer.from_pretrained('saibo/legal-longformer-base-4096')
    dbert_model = TFLongformerModel.from_pretrained('saibo/legal-longformer-base-4096')
    max_len=4096
    sentences=summarized_data['text']
    labels=summarized_data['label']
    len(sentences),len(labels)
    model_0=create_model()
    input_ids=[]
    attention_masks=[]

    for sent in sentences:
      dbert_inps=dbert_tokenizer.encode_plus(sent,add_special_tokens = True,max_length =max_len,pad_to_max_length = True,return_attention_mask = True,truncation=True)
      input_ids.append(dbert_inps['input_ids'])
      attention_masks.append(dbert_inps['attention_mask'])
    input_ids=np.asarray(input_ids)

    attention_masks=np.array(attention_masks)
    labels=np.array(labels)
    train_inp,val_inp,train_label,val_label,train_mask,val_mask=train_test_split(input_ids,labels,attention_masks,test_size=0.1,random_state=42)
    print(train_inp.shape)
    print(train_mask.shape)
    print(train_label.shape)
    print(val_inp.shape)
    print(val_mask.shape)
    print(val_label.shape)
    train_inp=train_inp[0:7576]
    train_mask=train_mask[0:7576]
    train_label=train_label[0:7576]

    val_inp=val_inp[0:840]
    val_label=val_label[0:840]
    val_mask=val_mask[0:840]
    print(train_inp.shape)
    print(train_mask.shape)
    print(train_label.shape)
    print(val_inp.shape)
    print(val_mask.shape)
    print(val_label.shape)
    log_dir='dbert_model'

    model_save_path='LSM/legallongformer-3-279labels.h5'

    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    accuracy = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')

    optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
    callbacks= [tf.keras.callbacks.ModelCheckpoint(filepath=model_save_path,monitor='val_accuracy',mode='max',save_best_only=True,save_weights_only=True),keras.callbacks.TensorBoard(log_dir=log_dir)]
    model_0.compile(loss=loss,optimizer=optimizer, metrics=[accuracy])

In [16]:
gc.collect()
tf.keras.backend.clear_session()
history=model_0.fit([train_inp,train_mask],train_label,batch_size=8,epochs=5,validation_data=([val_inp,val_mask],val_label),callbacks=callbacks)

2022-09-15 14:11:13.239681: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:766] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Did not find a shardable source, walked to a node which is not a dataset: name: "FlatMapDataset/_9"
op: "FlatMapDataset"
input: "PrefetchDataset/_8"
attr {
  key: "Targuments"
  value {
    list {
    }
  }
}
attr {
  key: "_cardinality"
  value {
    i: -2
  }
}
attr {
  key: "f"
  value {
    func {
      name: "__inference_Dataset_flat_map_slice_batch_indices_735406"
    }
  }
}
attr {
  key: "metadata"
  value {
    s: "\n\024FlatMapDataset:31162"
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: 8
        }
      }
    }
  }
}
attr {
  key: "output_types"
  value {
    list {
      type: DT_INT64
    }
  }
}
. Consider either turning off auto-sharding or switching the auto_shard_policy to DATA to shard this dataset.

Epoch 1/5


/ext3/miniconda3/envs/wol_env/lib/python3.7/site-packages/tensorflow/python/util/dispatch.py:1096: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


947/947 [==============================] - ETA: 0s - loss: 12.6649 - accuracy: 0.1587

2022-09-15 14:27:45.046694: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:766] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Did not find a shardable source, walked to a node which is not a dataset: name: "FlatMapDataset/_9"
op: "FlatMapDataset"
input: "PrefetchDataset/_8"
attr {
  key: "Targuments"
  value {
    list {
    }
  }
}
attr {
  key: "_cardinality"
  value {
    i: -2
  }
}
attr {
  key: "f"
  value {
    func {
      name: "__inference_Dataset_flat_map_slice_batch_indices_874546"
    }
  }
}
attr {
  key: "metadata"
  value {
    s: "\n\024FlatMapDataset:31187"
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: 8
        }
      }
    }
  }
}
attr {
  key: "output_types"
  value {
    list {
      type: DT_INT64
    }
  }
}
. Consider either turning off auto-sharding or switching the auto_shard_policy to DATA to shard this dataset.

947/947 [==============================] - 1052s 949ms/step - loss: 12.6649 - accuracy: 0.1587 - val_loss: 10.2974 - val_accuracy: 0.3643
Epoch 2/5
947/947 [==============================] - 874s 923ms/step - loss: 9.4930 - accuracy: 0.4011 - val_loss: 8.6128 - val_accuracy: 0.4857
Epoch 3/5
947/947 [==============================] - 874s 923ms/step - loss: 7.9735 - accuracy: 0.5174 - val_loss: 7.6501 - val_accuracy: 0.5083
Epoch 4/5
947/947 [==============================] - 874s 923ms/step - loss: 6.8458 - accuracy: 0.6113 - val_loss: 6.8636 - val_accuracy: 0.5417
Epoch 5/5
947/947 [==============================] - 874s 923ms/step - loss: 5.9304 - accuracy: 0.6843 - val_loss: 6.2744 - val_accuracy: 0.5762


In [15]:
pred_labels=[]

loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
accuracy = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)

model_saved= create_model()
model_saved.compile(loss=loss,optimizer=optimizer, metrics=[accuracy])
model_saved.load_weights('LSM/legallongformer-3-279labels.h5')

for i in range(0,len(val_inp)):
    pred=model_saved.predict([val_inp[i].reshape(1,4096),val_mask[i].reshape(1,4096)],batch_size=8)
    pred_label = pred.argmax(axis=1)
    pred_labels.append(pred_label)
accuracy=accuracy_score(val_label, pred_labels)
print("Accuracy: "+str(accuracy))
total_accuracy=total_accuracy+accuracy

weighted_f1=f1_score(val_label,pred_labels, average='weighted')
print("Weighted F1: "+ str(weighted_f1))
total_weighted_f1=total_weighted_f1+weighted_f1
micro_f1=f1_score(val_label,pred_labels, average='micro')
print("Micro F1: "+ str(micro_f1))
total_micro_f1=total_micro_f1+micro_f1

weighted_precision=precision_score(val_label, pred_labels, average='weighted')
print("Weighted Precision: " + str(weighted_precision))
total_weighted_precision=total_weighted_precision+weighted_precision
micro_precision=precision_score(val_label, pred_labels, average='micro')
print("Micro Precision: " + str(micro_precision))
total_micro_precision=total_micro_precision+micro_precision

weighted_recall=recall_score(val_label, pred_labels, average='weighted')
print("Weighted Recall: " + str(weighted_recall))
total_weighted_recall=total_weighted_recall+weighted_recall
micro_recall=recall_score(val_label, pred_labels, average='micro')
print("Micro Recall: " + str(micro_recall))
total_micro_recall=total_micro_recall+micro_recall

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_11 (InputLayer)          [(None, 4096)]       0           []                               
                                                                                                  
 input_12 (InputLayer)          [(None, 4096)]       0           []                               
                                                                                                  
 tf_longformer_model (TFLongfor  TFLongformerBaseMod  148659456  ['input_11[0][0]',               
 merModel)                      elOutputWithPooling               'input_12[0][0]']               
                                (last_hidden_state=                                               
                                (None, 4096, 768),                                          

/ext3/miniconda3/envs/wol_env/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/ext3/miniconda3/envs/wol_env/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
with strategy.scope():
    sc = SupremeCourt()
    print(sc.info)
    sc.download()

    texts = []  # list of text samples
    labels_index = {}  # dictionary mapping label name to numeric id
    labels = []  # list of label ids

    issue_codes = list(sc.issue_area_codes.keys()) # 15 labels
    print(issue_codes)
    issue_codes.sort()
    issue_codes = [str(ic) for ic in issue_codes]

    labels_index = dict(zip(issue_codes, np.arange(len(issue_codes))))
    print(labels_index)
    count=0

    for record in sc.records():

            count=count+1
            if record[1]['issue'] == None: # some cases have None as an issue
                labels.append(labels_index['-1'])
            else:
                labels.append(labels_index[record[1]['issue'][:-4]])

            new_sen=record[0].split("Footnotes")[0]
            new_new_sen=new_sen.split()
            if len(new_new_sen) >= 4096:
              new_new_sen=new_new_sen[0:4096]

            elif len(new_new_sen) < 4096:
              new_new_sen=new_new_sen[0:len(new_new_sen)]

            new_new_sen=' '.join(new_new_sen)

            texts.append(new_new_sen)

    len_list = [len(ele.split()) for ele in texts]

    #print(labels)
    print(len(labels))

    res = 0 if len(len_list) == 0 else (float(sum(len_list)) / len(len_list))
    print("Average Length %s" % res) 
    print('Found %s texts.' % len(texts))
    print('Found %s labels.' % len(labels_index))
    temp_file = open("labels_sc_279.txt", "r")

    data = temp_file.read()
    label_list = data.split("\n")
    #print(label_list)
    label_list = label_list[0:-1]
    #print(label_list)
    label_list = [int(i) for i in label_list]
    summarized_data = pd.DataFrame(texts,
                   columns =['text'])
    summarized_data['label'] = label_list
    print(summarized_data)
    #gc.collect()
    #tf.keras.backend.clear_session()
    dbert_tokenizer = LongformerTokenizer.from_pretrained('saibo/legal-longformer-base-4096')
    dbert_model = TFLongformerModel.from_pretrained('saibo/legal-longformer-base-4096')
    max_len=4096
    sentences=summarized_data['text']
    labels=summarized_data['label']
    len(sentences),len(labels)
    model_0=create_model()
    input_ids=[]
    attention_masks=[]

    for sent in sentences:
      dbert_inps=dbert_tokenizer.encode_plus(sent,add_special_tokens = True,max_length =max_len,pad_to_max_length = True,return_attention_mask = True,truncation=True)
      input_ids.append(dbert_inps['input_ids'])
      attention_masks.append(dbert_inps['attention_mask'])
    input_ids=np.asarray(input_ids)

    attention_masks=np.array(attention_masks)
    labels=np.array(labels)
    train_inp,val_inp,train_label,val_label,train_mask,val_mask=train_test_split(input_ids,labels,attention_masks,test_size=0.1,random_state=42)
    print(train_inp.shape)
    print(train_mask.shape)
    print(train_label.shape)
    print(val_inp.shape)
    print(val_mask.shape)
    print(val_label.shape)
    train_inp=train_inp[0:7576]
    train_mask=train_mask[0:7576]
    train_label=train_label[0:7576]

    val_inp=val_inp[0:840]
    val_label=val_label[0:840]
    val_mask=val_mask[0:840]
    print(train_inp.shape)
    print(train_mask.shape)
    print(train_label.shape)
    print(val_inp.shape)
    print(val_mask.shape)
    print(val_label.shape)
    log_dir='dbert_model'

    model_save_path='LSM/legallongformer-4-279labels.h5'

    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    accuracy = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')

    optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
    callbacks= [tf.keras.callbacks.ModelCheckpoint(filepath=model_save_path,monitor='val_accuracy',mode='max',save_best_only=True,save_weights_only=True),keras.callbacks.TensorBoard(log_dir=log_dir)]
    model_0.compile(loss=loss,optimizer=optimizer, metrics=[accuracy])

In [ ]:
gc.collect()
tf.keras.backend.clear_session()
history=model_0.fit([train_inp,train_mask],train_label,batch_size=8,epochs=5,validation_data=([val_inp,val_mask],val_label),callbacks=callbacks)

2022-09-15 15:32:30.468403: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:766] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Did not find a shardable source, walked to a node which is not a dataset: name: "FlatMapDataset/_9"
op: "FlatMapDataset"
input: "PrefetchDataset/_8"
attr {
  key: "Targuments"
  value {
    list {
    }
  }
}
attr {
  key: "_cardinality"
  value {
    i: -2
  }
}
attr {
  key: "f"
  value {
    func {
      name: "__inference_Dataset_flat_map_slice_batch_indices_1079788"
    }
  }
}
attr {
  key: "metadata"
  value {
    s: "\n\024FlatMapDataset:42166"
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: 8
        }
      }
    }
  }
}
attr {
  key: "output_types"
  value {
    list {
      type: DT_INT64
    }
  }
}
. Consider either turning off auto-sharding or switching the auto_shard_policy to DATA to shard this dataset

Epoch 1/5


/ext3/miniconda3/envs/wol_env/lib/python3.7/site-packages/tensorflow/python/util/dispatch.py:1096: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


947/947 [==============================] - ETA: 0s - loss: 12.6415 - accuracy: 0.1521

2022-09-15 15:49:01.542147: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:766] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Did not find a shardable source, walked to a node which is not a dataset: name: "FlatMapDataset/_9"
op: "FlatMapDataset"
input: "PrefetchDataset/_8"
attr {
  key: "Targuments"
  value {
    list {
    }
  }
}
attr {
  key: "_cardinality"
  value {
    i: -2
  }
}
attr {
  key: "f"
  value {
    func {
      name: "__inference_Dataset_flat_map_slice_batch_indices_1218928"
    }
  }
}
attr {
  key: "metadata"
  value {
    s: "\n\024FlatMapDataset:42191"
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: 8
        }
      }
    }
  }
}
attr {
  key: "output_types"
  value {
    list {
      type: DT_INT64
    }
  }
}
. Consider either turning off auto-sharding or switching the auto_shard_policy to DATA to shard this dataset

947/947 [==============================] - 1053s 950ms/step - loss: 12.6415 - accuracy: 0.1521 - val_loss: 10.3385 - val_accuracy: 0.3417
Epoch 2/5
808/947 [========================>.....] - ETA: 2:02 - loss: 9.5539 - accuracy: 0.3874

In [16]:
pred_labels=[]

loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
accuracy = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)

model_saved= create_model()
model_saved.compile(loss=loss,optimizer=optimizer, metrics=[accuracy])
model_saved.load_weights('LSM/legallongformer-4-279labels.h5')

for i in range(0,len(val_inp)):
    pred=model_saved.predict([val_inp[i].reshape(1,4096),val_mask[i].reshape(1,4096)],batch_size=8)
    pred_label = pred.argmax(axis=1)
    pred_labels.append(pred_label)
accuracy=accuracy_score(val_label, pred_labels)
print("Accuracy: "+str(accuracy))
total_accuracy=total_accuracy+accuracy

weighted_f1=f1_score(val_label,pred_labels, average='weighted')
print("Weighted F1: "+ str(weighted_f1))
total_weighted_f1=total_weighted_f1+weighted_f1
micro_f1=f1_score(val_label,pred_labels, average='micro')
print("Micro F1: "+ str(micro_f1))
total_micro_f1=total_micro_f1+micro_f1

weighted_precision=precision_score(val_label, pred_labels, average='weighted')
print("Weighted Precision: " + str(weighted_precision))
total_weighted_precision=total_weighted_precision+weighted_precision
micro_precision=precision_score(val_label, pred_labels, average='micro')
print("Micro Precision: " + str(micro_precision))
total_micro_precision=total_micro_precision+micro_precision

weighted_recall=recall_score(val_label, pred_labels, average='weighted')
print("Weighted Recall: " + str(weighted_recall))
total_weighted_recall=total_weighted_recall+weighted_recall
micro_recall=recall_score(val_label, pred_labels, average='micro')
print("Micro Recall: " + str(micro_recall))
total_micro_recall=total_micro_recall+micro_recall

Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_13 (InputLayer)          [(None, 4096)]       0           []                               
                                                                                                  
 input_14 (InputLayer)          [(None, 4096)]       0           []                               
                                                                                                  
 tf_longformer_model (TFLongfor  TFLongformerBaseMod  148659456  ['input_13[0][0]',               
 merModel)                      elOutputWithPooling               'input_14[0][0]']               
                                (last_hidden_state=                                               
                                (None, 4096, 768),                                          

/ext3/miniconda3/envs/wol_env/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/ext3/miniconda3/envs/wol_env/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [17]:
print("Average Accuracy: "+str(total_accuracy/5))
print("Average Weighted F1: "+str(total_weighted_f1/5))
print("Average Micro F1: "+str(total_micro_f1/5))
print("Average Weighted Precision: "+str(total_weighted_precision/5))
print("Average Micro Precision: "+str(total_micro_precision/5))
print("Average Weighted Recall: "+str(total_weighted_recall/5))
print("Average Micro Recall: "+str(total_micro_recall/5))

Average Accuracy: 0.5626190476190477
Average Weighted F1: 0.5197154877374754
Average Micro F1: 0.5626190476190477
Average Weighted Precision: 0.5158237907099823
Average Micro Precision: 0.5626190476190477
Average Weighted Recall: 0.5626190476190477
Average Micro Recall: 0.5626190476190477
